# Programming Assignment: 
## Готовим LDA по рецептам

Как вы уже знаете, в тематическом моделировании делается предположение о том, что для определения тематики порядок слов в документе не важен; об этом гласит гипотеза «мешка слов». Сегодня мы будем работать с несколько нестандартной для тематического моделирования коллекцией, которую можно назвать «мешком ингредиентов», потому что на состоит из рецептов блюд разных кухонь. Тематические модели ищут слова, которые часто вместе встречаются в документах, и составляют из них темы. Мы попробуем применить эту идею к рецептам и найти кулинарные «темы». Эта коллекция хороша тем, что не требует предобработки. Кроме того, эта задача достаточно наглядно иллюстрирует принцип работы тематических моделей.

Для выполнения заданий, помимо часто используемых в курсе библиотек, потребуются модули *json* и *gensim*. Первый входит в дистрибутив Anaconda, второй можно поставить командой 

*pip install gensim*

Построение модели занимает некоторое время. На ноутбуке с процессором Intel Core i7 и тактовой частотой 2400 МГц на построение одной модели уходит менее 10 минут.

### Загрузка данных

Коллекция дана в json-формате: для каждого рецепта известны его id, кухня (cuisine) и список ингредиентов, в него входящих. Загрузить данные можно с помощью модуля json (он входит в дистрибутив Anaconda):

In [94]:
import json

In [95]:
with open("recipes.json") as f:
    recipes = json.load(f)

In [96]:
print recipes[:3]

[{u'cuisine': u'greek', u'id': 10259, u'ingredients': [u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']}, {u'cuisine': u'southern_us', u'id': 25693, u'ingredients': [u'plain flour', u'ground pepper', u'salt', u'tomatoes', u'ground black pepper', u'thyme', u'eggs', u'green tomatoes', u'yellow corn meal', u'milk', u'vegetable oil']}, {u'cuisine': u'filipino', u'id': 20130, u'ingredients': [u'eggs', u'pepper', u'salt', u'mayonaise', u'cooking oil', u'green chilies', u'grilled chicken breasts', u'garlic powder', u'yellow onion', u'soy sauce', u'butter', u'chicken livers']}]


### Составление корпуса

In [97]:
from gensim import corpora, models
import numpy as np

Наша коллекция небольшая, и целиком помещается в оперативную память. Gensim может работать с такими данными и не требует их сохранения на диск в специальном формате. Для этого коллекция должна быть представлена в виде списка списков, каждый внутренний список соответствует отдельному документу и состоит из его слов. Пример коллекции из двух документов: 

[["hello", "world"], ["programming", "in", "python"]]

Преобразуем наши данные в такой формат, а затем создадим объекты corpus и dictionary, с которыми будет работать модель.

In [98]:
texts = [recipe["ingredients"] for recipe in recipes]
dictionary = corpora.Dictionary(texts)   # составляем словарь
corpus = [dictionary.doc2bow(text) for text in texts]  # составляем корпус документов

In [6]:
print texts[0]
print corpus[0]

[u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]


In [7]:
# create dictionary with id to token
inv_dict = {v: k for k, v in dictionary.token2id.iteritems()}

In [8]:
dictionary.token2id['sweetened coconut']

4351

In [9]:
inv_dict[116]

u'cooking spray'

У объекта dictionary есть полезная переменная dictionary.token2id, позволяющая находить соответствие между ингредиентами и их индексами.

### Обучение модели
Вам может понадобиться [документация](https://radimrehurek.com/gensim/models/ldamodel.html) LDA в gensim.

__Задание 1.__ Обучите модель LDA с 40 темами, установив количество проходов по коллекции 5 и оставив остальные параметры по умолчанию. 


Затем вызовите метод модели *show_topics*, указав количество тем 40 и количество токенов 10, и сохраните результат (топы ингредиентов в темах) в отдельную переменную. Если при вызове метода *show_topics* указать параметр *formatted=True*, то топы ингредиентов будет удобно выводить на печать, если *formatted=False*, будет удобно работать со списком программно. Выведите топы на печать, рассмотрите темы, а затем ответьте на вопрос:

Сколько раз ингредиенты "salt", "sugar", "water", "mushrooms", "chicken", "eggs" встретились среди топов-10 всех 40 тем? При ответе __не нужно__ учитывать составные ингредиенты, например, "hot water".

Передайте 6 чисел в функцию save_answers1 и загрузите сгенерированный файл в форму.

У gensim нет возможности фиксировать случайное приближение через параметры метода, но библиотека использует numpy для инициализации матриц. Поэтому, по утверждению автора библиотеки, фиксировать случайное приближение нужно командой, которая написана в следующей ячейке. __Перед строкой кода с построением модели обязательно вставляйте указанную строку фиксации random.seed.__

In [51]:
print np.__version__

1.11.3


In [53]:
# gensim 0.13.4.1
#print gensim.__verison__

In [76]:
%%time
np.random.seed(76543)
# здесь код для построения модели:
#lda = models.ldamodel.LdaModel(corpus=corpus, id2word = dictionary, num_topics=40, passes = 5)
lda = models.LdaModel(corpus=corpus, id2word = dictionary, num_topics=40, passes = 5)

CPU times: user 6min 17s, sys: 35.8 ms, total: 6min 17s
Wall time: 6min 17s


In [88]:
np.random.seed(76543) 
#tupleTopics = lda.show_topics(num_words=10, num_topics=40, log=False, formatted=False)
tupleTopics = lda.show_topics(num_topics=40, num_words=10, log=False, formatted=False)

In [89]:
tupleTopics[0]

(0,
 [(u'cooking spray', 0.08893168359335854),
  (u'salt', 0.083023313122701475),
  (u'garlic cloves', 0.07969448238353348),
  (u'olive oil', 0.068056697732332527),
  (u'chopped onion', 0.065740286303090784),
  (u'crushed red pepper', 0.03660342538942804),
  (u'fat free less sodium chicken broth', 0.036151588839948151),
  (u'black pepper', 0.034352059579260526),
  (u'ground black pepper', 0.032028828039853005),
  (u'water', 0.032026346546356485)])

In [90]:
response_dict = { "salt":0, "sugar":0, "water":0, "mushrooms":0, "chicken":0, "eggs":0}

In [91]:
for topic in tupleTopics:
    #print topic[0]
    for token in topic[1]:
        #ingredient = inv_dict[int(token[0])]
        ingredient = token[0]
        if ingredient in response_dict.keys():
            print ingredient, token[1]
            response_dict[ingredient] = response_dict[ingredient] + 1

salt 0.0830233131227
water 0.0320263465464
water 0.0325720540325
salt 0.0633267330439
salt 0.053129801327
salt 0.052261476695
salt 0.0392146808301
salt 0.0672771355352
water 0.0587491899223
salt 0.0525761998909
salt 0.0240673118177
salt 0.064211043653
sugar 0.0242324199933
sugar 0.0443938891923
salt 0.0380800601569
salt 0.0196755431489
salt 0.102864758894
eggs 0.0865729159299
sugar 0.0497743149661
sugar 0.0824532185208
water 0.0289495451231
salt 0.0658717534433
sugar 0.0418488645069
water 0.0303681915686
salt 0.0558526938849
mushrooms 0.133700626002
sugar 0.0667066361242
salt 0.0574875586166
salt 0.0713393508017
water 0.0521447105911
water 0.0628778181516
salt 0.0739192259515
chicken 0.0316418028852
sugar 0.0421829654498
salt 0.028955740531
water 0.0284877279029
salt 0.0347840827265
salt 0.0542070152872


In [92]:
response_dict

{'chicken': 1, 'eggs': 1, 'mushrooms': 1, 'salt': 20, 'sugar': 7, 'water': 8}

In [82]:
c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs = (0,0,0,0,0,0)

In [83]:
import re

In [84]:
resQuant = []
for c_var in ['c_salt', 'c_sugar', 'c_water', 'c_mushrooms', 'c_chicken','c_eggs']:
    print c_var, response_dict[re.sub('^c_', '', c_var)]
    resQuant.append(response_dict[re.sub('^c_', '', c_var)])
resQuant

c_salt 20
c_sugar 7
c_water 8
c_mushrooms 1
c_chicken 1
c_eggs 1


[20, 7, 8, 1, 1, 1]

In [85]:
c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs = resQuant[:]

In [86]:
def save_answers1(c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs):
    with open("cooking_LDA_pa_task1.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs]]))

In [87]:
save_answers1(c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs)

### Фильтрация словаря
В топах тем гораздо чаще встречаются первые три рассмотренных ингредиента, чем последние три. При этом наличие в рецепте курицы, яиц и грибов яснее дает понять, что мы будем готовить, чем наличие соли, сахара и воды. Таким образом, даже в рецептах есть слова, часто встречающиеся в текстах и не несущие смысловой нагрузки, и поэтому их не желательно видеть в темах. Наиболее простой прием борьбы с такими фоновыми элементами — фильтрация словаря по частоте. Обычно словарь фильтруют с двух сторон: убирают очень редкие слова (в целях экономии памяти) и очень частые слова (в целях повышения интерпретируемости тем). Мы уберем только частые слова.

In [99]:
import copy
dictionary2 = copy.deepcopy(dictionary)

__Задание 2.__ У объекта dictionary2 есть переменная *dfs* — это словарь, ключами которого являются id токена, а элементами — число раз, сколько слово встретилось во всей коллекции. Сохраните в отдельный список ингредиенты, которые встретились в коллекции больше 4000 раз. Вызовите метод словаря *filter_tokens*, подав в качестве первого аргумента полученный список популярных ингредиентов. Вычислите две величины: dict_size_before и dict_size_after — размер словаря до и после фильтрации.

Затем, используя новый словарь, создайте новый корпус документов, corpus2, по аналогии с тем, как это сделано в начале ноутбука. Вычислите две величины: corpus_size_before и corpus_size_after — суммарное количество ингредиентов в корпусе (для каждого документа вычислите число различных ингредиентов в нем и просуммируйте по всем документам) до и после фильтрации.

Передайте величины dict_size_before, dict_size_after, corpus_size_before, corpus_size_after в функцию save_answers2 и загрузите сгенерированный файл в форму.

In [60]:
dictionary2.dfs

{0: 4438,
 1: 137,
 2: 148,
 3: 228,
 4: 7380,
 5: 229,
 6: 358,
 7: 270,
 8: 1896,
 9: 4385,
 10: 154,
 11: 361,
 12: 4784,
 13: 3388,
 14: 341,
 15: 108,
 16: 385,
 17: 18048,
 18: 2263,
 19: 3058,
 20: 1442,
 21: 4847,
 22: 768,
 23: 483,
 24: 65,
 25: 781,
 26: 3296,
 27: 5,
 28: 1184,
 29: 7457,
 30: 26,
 31: 103,
 32: 839,
 33: 2036,
 34: 343,
 35: 18,
 36: 282,
 37: 1394,
 38: 2627,
 39: 1523,
 40: 24,
 41: 1970,
 42: 40,
 43: 1477,
 44: 925,
 45: 7972,
 46: 2747,
 47: 338,
 48: 6434,
 49: 1738,
 50: 501,
 51: 1231,
 52: 678,
 53: 1298,
 54: 7971,
 55: 103,
 56: 461,
 57: 554,
 58: 1730,
 59: 1094,
 60: 796,
 61: 73,
 62: 940,
 63: 1348,
 64: 1,
 65: 173,
 66: 22,
 67: 51,
 68: 300,
 69: 424,
 70: 164,
 71: 965,
 72: 119,
 73: 1698,
 74: 1439,
 75: 56,
 76: 1137,
 77: 298,
 78: 3113,
 79: 2747,
 80: 62,
 81: 1223,
 82: 134,
 83: 1706,
 84: 578,
 85: 185,
 86: 164,
 87: 55,
 88: 154,
 89: 439,
 90: 1503,
 91: 505,
 92: 425,
 93: 168,
 94: 1891,
 95: 1773,
 96: 264,
 97: 341,
 98:

In [67]:
%%time
# Сохраните в отдельный список ингредиенты, которые встретились в коллекции больше 4000 раз.
d = {k: v for k, v in dictionary2.dfs.items() if v > 4000}

CPU times: user 1.6 ms, sys: 0 ns, total: 1.6 ms
Wall time: 1.6 ms


In [68]:
len(d.keys())

12

In [69]:
d

{0: 4438,
 4: 7380,
 9: 4385,
 12: 4784,
 17: 18048,
 21: 4847,
 29: 7457,
 45: 7972,
 48: 6434,
 54: 7971,
 100: 6236,
 117: 4632}

In [52]:
{k: v for k, v in dictionary.items() if v > 4000}

{0: u'pepper',
 1: u'seasoning',
 2: u'garbanzo beans',
 3: u'grape tomatoes',
 4: u'garlic',
 5: u'black olives',
 6: u'feta cheese crumbles',
 7: u'romaine lettuce',
 8: u'purple onion',
 9: u'vegetable oil',
 10: u'plain flour',
 11: u'thyme',
 12: u'ground black pepper',
 13: u'eggs',
 14: u'yellow corn meal',
 15: u'green tomatoes',
 16: u'ground pepper',
 17: u'salt',
 18: u'milk',
 19: u'tomatoes',
 20: u'garlic powder',
 21: u'butter',
 22: u'green chilies',
 23: u'cooking oil',
 24: u'chicken livers',
 25: u'mayonaise',
 26: u'soy sauce',
 27: u'grilled chicken breasts',
 28: u'yellow onion',
 29: u'water',
 30: u'wheat',
 31: u'cornflour',
 32: u'bay leaf',
 33: u'chili powder',
 34: u'boneless chicken skinless thigh',
 35: u'natural yogurt',
 36: u'garlic paste',
 37: u'lemon juice',
 38: u'black pepper',
 39: u'cayenne pepper',
 40: u'passata',
 41: u'oil',
 42: u'double cream',
 43: u'shallots',
 44: u'garam masala',
 45: u'onions',
 46: u'ground cumin',
 47: u'fresh ginge

In [70]:
# Вызовите метод словаря filter_tokens, подав в качестве первого аргумента полученный список популярных ингредиентов. 
# Вычислите две величины: dict_size_before и dict_size_after — размер словаря до и после фильтрации.

dictionary2.filter_tokens(d)

In [81]:
dict_size_before = len(dictionary.token2id)

In [82]:
dict_size_after = len(dictionary2.token2id)

In [ ]:
# Затем, используя новый словарь, создайте новый корпус документов, corpus2, 
# по аналогии с тем, как это сделано в начале ноутбука. 
# Вычислите две величины: corpus_size_before и corpus_size_after — суммарное количество ингредиентов в корпусе 
# (для каждого документа вычислите число различных ингредиентов в нем и просуммируйте по всем документам) до и после фильтрации.

In [100]:
corpus2 = [dictionary2.doc2bow(text) for text in texts]  # составляем корпус документов

In [94]:
corpus2[10]

[(530, 1),
 (1052, 1),
 (1553, 1),
 (1761, 1),
 (2840, 1),
 (3527, 1),
 (3642, 1),
 (4405, 1),
 (4851, 1),
 (6415, 1)]

In [93]:
corpus[10]

[(0, 1),
 (4, 1),
 (5, 1),
 (54, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1)]

In [90]:
dictionary2.token2id['salt']

KeyError: 'salt'

In [ ]:
# (для каждого документа вычислите число различных ингредиентов в нем и просуммируйте по всем документам) до и после фильтрации.

In [98]:
def calcSize(corpus):
    sumTokens = 0
    for tText in corpus:
        sumTokens = sumTokens + len(tText)
    return sumTokens

In [99]:
corpus_size_before = calcSize(corpus)
corpus_size_after = calcSize(corpus2)

In [101]:
print corpus_size_before, corpus_size_after

428249 343665


In [103]:
def save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after):
    with open("cooking_LDA_pa_task2.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [dict_size_before, dict_size_after, corpus_size_before, corpus_size_after]]))

In [104]:
save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after)

### Сравнение когерентностей
__Задание 3.__ Постройте еще одну модель по корпусу corpus2 и словарю dictionary2, остальные параметры оставьте такими же, как при первом построении модели. Сохраните новую модель в другую переменную (не перезаписывайте предыдущую модель). Не забудьте про фиксирование seed!

Затем воспользуйтесь методом *top_topics* модели, чтобы вычислить ее когерентность. Передайте в качестве аргумента соответствующий модели корпус. Метод вернет список кортежей (топ токенов, когерентность), отсортированных по убыванию последней. Вычислите среднюю по всем темам когерентность для каждой из двух моделей и передайте в функцию save_answers3. 

In [101]:
%%time
np.random.seed(76543)
# здесь код для построения модели:
lda2 = models.ldamodel.LdaModel(corpus=corpus2, num_topics=40, passes=5)

CPU times: user 6min 35s, sys: 27.8 ms, total: 6min 35s
Wall time: 6min 35s


In [102]:
tt2 = lda2.top_topics(corpus = corpus2)

In [103]:
%%time
np.random.seed(76543)
# здесь код для построения модели:
lda = models.ldamodel.LdaModel(corpus=corpus, num_topics=40, passes=5)

CPU times: user 8min 8s, sys: 116 ms, total: 8min 8s
Wall time: 8min 9s


In [104]:
tt = lda.top_topics(corpus = corpus2)

In [108]:
type(tt)

list

In [112]:
len(tt)

40

In [118]:
tt[0][1]

-502.29335469567718

In [106]:
def calcMeanCoh(tt):
    cohSum = 0
    cohNum = len(tt)
    for topic in tt:
        cohSum = cohSum + topic[1]
    return 1.0*cohSum/cohNum

In [109]:
coh1 = calcMeanCoh(tt)

In [110]:
coh2 = calcMeanCoh(tt2)

In [128]:
tt[0][0]

[(0.055272222505536248, '6154'),
 (0.053784369720157342, '3241'),
 (0.050578416789333953, '3424'),
 (0.045760713532694508, '1380'),
 (0.041568358741365864, '1351'),
 (0.038384499214146675, '4032'),
 (0.034853629564375731, '1071'),
 (0.034328002361357121, '4043'),
 (0.033296165712938308, '5019'),
 (0.032528822251177383, '6583'),
 (0.023917627129996309, '1809'),
 (0.019610235119811376, '3114'),
 (0.019461786747636864, '4011'),
 (0.016685664396463885, '2321'),
 (0.016220949102322874, '5106'),
 (0.015600479126117167, '1327'),
 (0.015538932062850766, '4893'),
 (0.015091405656480966, '1718'),
 (0.01484088989032621, '5101'),
 (0.013980189485417472, '164')]

In [130]:
tt2[0][0]

[(0.055272222505536248, '6154'),
 (0.053784369720157342, '3241'),
 (0.050578416789333953, '3424'),
 (0.045760713532694508, '1380'),
 (0.041568358741365864, '1351'),
 (0.038384499214146675, '4032'),
 (0.034853629564375731, '1071'),
 (0.034328002361357121, '4043'),
 (0.033296165712938308, '5019'),
 (0.032528822251177383, '6583'),
 (0.023917627129996309, '1809'),
 (0.019610235119811376, '3114'),
 (0.019461786747636864, '4011'),
 (0.016685664396463885, '2321'),
 (0.016220949102322874, '5106'),
 (0.015600479126117167, '1327'),
 (0.015538932062850766, '4893'),
 (0.015091405656480966, '1718'),
 (0.01484088989032621, '5101'),
 (0.013980189485417472, '164')]

In [ ]:
# Вычислите среднюю по всем темам когерентность для каждой из двух моделей и передайте в функцию save_answers3. 

In [137]:
def calcMeanCoherence(tt):
    meanCoherence = 0
    i = 0
    sumCoh = 0
    for topic in tt:
        for word in topic[0]:
            #print word
            sumCoh = sumCoh + word[0]
            i = i+1
    meanCoherence = 1.0 * sumCoh / i
    return meanCoherence

In [138]:
coherence2 = calcMeanCoherence(tt2)
print coherence2

0.0338317631285


In [139]:
tt = lda.top_topics(corpus = corpus)

In [140]:
coherence = calcMeanCoherence(tt)
print coherence

0.0331683011291


In [112]:
def save_answers3(coherence, coherence2):
    with open("cooking_LDA_pa_task3.txt", "w") as fout:
        fout.write(" ".join(["%3f"%el for el in [coherence, coherence2]]))

In [113]:
save_answers3(coh1, coh2)

Считается, что когерентность хорошо соотносится с человеческими оценками интерпретируемости тем. Поэтому на больших текстовых коллекциях когерентность обычно повышается, если убрать фоновую лексику. Однако в нашем случае этого не произошло. 

### Изучение влияния гиперпараметра alpha

В этом разделе мы будем работать со второй моделью, то есть той, которая построена по сокращенному корпусу. 

Пока что мы посмотрели только на матрицу темы-слова, теперь давайте посмотрим на матрицу темы-документы. Выведите темы для нулевого (или любого другого) документа из корпуса, воспользовавшись методом *get_document_topics* второй модели:

Также выведите содержимое переменной *.alpha* второй модели:

У вас должно получиться, что документ характеризуется небольшим числом тем. Попробуем поменять гиперпараметр alpha, задающий априорное распределение Дирихле для распределений тем в документах.

__Задание 4.__ Обучите третью модель: используйте сокращенный корпус (corpus2 и dictionary2) и установите параметр __alpha=1__, passes=5. Не забудьте про фиксацию seed! Выведите темы новой модели для нулевого документа; должно получиться, что распределение над множеством тем практически равномерное. Чтобы убедиться в том, что во второй модели документы описываются гораздо более разреженными распределениями, чем в третьей, посчитайте суммарное количество элементов, __превосходящих 0.01__, в матрицах темы-документы обеих моделей. Другими словами, запросите темы  модели для каждого документа с параметром *minimum_probability=0.01* и просуммируйте число элементов в получаемых массивах. Передайте две суммы (сначала для модели с alpha по умолчанию, затем для модели в alpha=1) в функцию save_answers4.

In [ ]:
def save_answers4(count_model2, count_model3):
    with open("cooking_LDA_pa_task4.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [count_model2, count_model3]]))

Таким образом, гиперпараметр __alpha__ влияет на разреженность распределений тем в документах. Аналогично гиперпараметр __eta__ влияет на разреженность распределений слов в темах.

### LDA как способ понижения размерности
Иногда, распределения над темами, найденные с помощью LDA, добавляют в матрицу объекты-признаки как дополнительные, семантические, признаки, и это может улучшить качество решения задачи. Для простоты давайте просто обучим классификатор рецептов на кухни на признаках, полученных из LDA, и измерим точность (accuracy).

__Задание 5.__ Используйте модель, построенную по сокращенной выборке с alpha по умолчанию (вторую модель). Составьте матрицу $\Theta = p(t|d)$ вероятностей тем в документах; вы можете использовать тот же метод get_document_topics, а также вектор правильных ответов y (в том же порядке, в котором рецепты идут в переменной recipes). Создайте объект RandomForestClassifier со 100 деревьями, с помощью функции cross_val_score вычислите среднюю accuracy по трем фолдам (перемешивать данные не нужно) и передайте в функцию save_answers5.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

In [ ]:
def save_answers5(accuracy):
     with open("cooking_LDA_pa_task5.txt", "w") as fout:
        fout.write(str(accuracy))

Для такого большого количества классов это неплохая точность. Вы можете попроовать обучать RandomForest на исходной матрице частот слов, имеющей значительно большую размерность, и увидеть, что accuracy увеличивается на 10–15%. Таким образом, LDA собрал не всю, но достаточно большую часть информации из выборки, в матрице низкого ранга.

### LDA — вероятностная модель
Матричное разложение, использующееся в LDA, интерпретируется как следующий процесс генерации документов.

Для документа $d$ длины $n_d$:
1. Из априорного распределения Дирихле с параметром alpha сгенерировать распределение над множеством тем: $\theta_d \sim Dirichlet(\alpha)$
1. Для каждого слова $w = 1, \dots, n_d$:
    1. Сгенерировать тему из дискретного распределения $t \sim \theta_{d}$
    1. Сгенерировать слово из дискретного распределения $w \sim \phi_{t}$.
    
Подробнее об этом в [Википедии](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation).

В контексте нашей задачи получается, что, используя данный генеративный процесс, можно создавать новые рецепты. Вы можете передать в функцию модель и число ингредиентов и сгенерировать рецепт :)

In [ ]:
def generate_recipe(model, num_ingredients):
    theta = np.random.dirichlet(model.alpha)
    for i in range(num_ingredients):
        t = np.random.choice(np.arange(model.num_topics), p=theta)
        topic = model.show_topic(t, topn=model.num_terms)
        topic_distr = [x[1] for x in topic]
        terms = [x[0] for x in topic]
        w = np.random.choice(terms, p=topic_distr)
        print w

### Интерпретация построенной модели
Вы можете рассмотреть топы ингредиентов каждой темы. Большиснтво тем сами по себе похожи на рецепты; в некоторых собираются продукты одного вида, например, свежие фрукты или разные виды сыра.

Попробуем эмпирически соотнести наши темы с национальными кухнями (cuisine). Построим матрицу $A$ размера темы $x$ кухни, ее элементы $a_{tc}$ — суммы $p(t|d)$ по всем документам $d$, которые отнесены к кухне $c$. Нормируем матрицу на частоты рецептов по разным кухням, чтобы избежать дисбаланса между кухнями. Следующая функция получает на вход объект модели, объект корпуса и исходные данные и возвращает нормированную матрицу $A$. Ее удобно визуализировать с помощью seaborn.

In [ ]:
import pandas
import seaborn
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def compute_topic_cuisine_matrix(model, corpus, recipes):
    # составляем вектор целевых признаков
    targets = list(set([recipe["cuisine"] for recipe in recipes]))
    # составляем матрицу
    tc_matrix = pandas.DataFrame(data=np.zeros((model.num_topics, len(targets))), columns=targets)
    for recipe, bow in zip(recipes, corpus):
        recipe_topic = model.get_document_topics(bow)
        for t, prob in recipe_topic:
            tc_matrix[recipe["cuisine"]][t] += prob
    # нормируем матрицу
    target_sums = pandas.DataFrame(data=np.zeros((1, len(targets))), columns=targets)
    for recipe in recipes:
        target_sums[recipe["cuisine"]] += 1
    return pandas.DataFrame(tc_matrix.values/target_sums.values, columns=tc_matrix.columns)

In [ ]:
def plot_matrix(tc_matrix):
    plt.figure(figsize=(10, 10))
    seaborn.heatmap(tc_matrix, square=True)

In [ ]:
# Визуализируйте матрицу


Чем темнее квадрат в матрице, тем больше связь этой темы с данной кухней. Мы видим, что у нас есть темы, которые связаны с несколькими кухнями. Такие темы показывают набор ингредиентов, которые популярны в кухнях нескольких народов, то есть указывают на схожесть кухонь этих народов. Некоторые темы распределены по всем кухням равномерно, они показывают наборы продуктов, которые часто используются в кулинарии всех стран. 

Жаль, что в датасете нет названий рецептов, иначе темы было бы проще интерпретировать...

### Заключение
В этом задании вы построили несколько моделей LDA, посмотрели, на что влияют гиперпараметры модели и как можно использовать построенную модель. 